In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 1. Load NMOS Data

In [ ]:
# NMOS Part 2: ID vs VDS for different VGS values (in Amperes)
VGS_values_nmos = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
VDS_values_nmos = np.array([0.04, 1, 4, 10])

# ID values in Amperes (columns: Vds=0.04, 1, 4, 10)
ID_nmos_part2 = np.array([
    [0, 0, 0, 0],      # VGS=0
    [0, 0, 0, 0],      # VGS=1
    [0, 0.004, 0.005, 0.001],  # VGS=2
    [0, 0.009, 0.013, 0.014],  # VGS=3
    [0, 0.014, 0.024, 0.025],  # VGS=4
    [0, 0.018, 0.036, 0.037],  # VGS=5
    [0, 0.021, 0.049, 0.050],  # VGS=6
    [0, 0.024, 0.062, 0.063],  # VGS=7
    [0, 0.027, 0.073, 0.075],  # VGS=8
    [0, 0.029, 0.083, 0.088],  # VGS=9
    [0, 0.032, 0.091, 0.100],  # VGS=10
])

# NMOS Part 3: ID vs VGS for different VDS values
VDS_values_nmos_part3 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
VGS_values_nmos_part3 = np.array([0.4, 1, 4, 10])

ID_nmos_part3 = np.array([
    [0, 0, 0, 0],          # Vds=0
    [0, 0, 0.014, 0.031],  # Vds=1
    [0, 0, 0.021, 0.057],  # Vds=2
    [0, 0, 0.023, 0.078],  # Vds=3
    [0, 0, 0.023, 0.093],  # Vds=4
    [0, 0, 0.023, 0.100],  # Vds=5
    [0, 0, 0.023, 0.100],  # Vds=6
    [0, 0, 0.023, 0.100],  # Vds=7
    [0, 0, 0.023, 0.100],  # Vds=8
    [0, 0, 0.023, 0.100],  # Vds=9
    [0, 0, 0.024, 0.100],  # Vds=10
])

print("NMOS data loaded successfully")
print(f"ID at VDS=10V: {ID_nmos_part2[:, 3]}")

## 2. Load PMOS Data (with interpolated VDS=4V values)

In [ ]:
# PMOS Part 2: ID vs VDS for different VGS values
VGS_values_pmos = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
VDS_values_pmos = np.array([0.04, 1, 4, 10])

# PMOS data with interpolated VDS=4V column
# Interpolating between VDS=1V and VDS=10V for missing VDS=4V data
ID_pmos_vds_004 = np.array([0, 0, 0.001, 0.002, 0.005, 0.007, 0.008, 0.008, 0.008, 0.008, 0.007])
ID_pmos_vds_1 = np.array([0, 0, 0.001, 0.003, 0.005, 0.008, 0.008, 0.008, 0.008, 0.008, 0.008])
ID_pmos_vds_10 = np.array([0.008, 0.008, 0.008, 0.008, 0.008, 0.009, 0.012, 0.014, 0.017, 0.019, 0.022])

# Linear interpolation for VDS=4V (weighted average)
# VDS=4 is between 1 and 10, so weight: (4-1)/(10-1) = 3/9 = 1/3
ID_pmos_vds_4 = ID_pmos_vds_1 * (2/3) + ID_pmos_vds_10 * (1/3)

ID_pmos_part2 = np.column_stack([ID_pmos_vds_004, ID_pmos_vds_1, ID_pmos_vds_4, ID_pmos_vds_10])

print("PMOS Part 2 data with interpolated VDS=4V:")
print(f"VDS=4V values: {ID_pmos_vds_4}")

# PMOS Part 3: ID vs VGS for different VDS values
VDS_values_pmos_part3 = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
VGS_values_pmos_part3 = np.array([0.4, 1, 4, 10])

# Similarly interpolate VGS=4V column for Part 3
ID_pmos_part3_vgs04 = np.array([0, 0, 0.001, 0.003, 0.005, 0.006, 0.007, 0.007, 0.007, 0.007, 0.007])
ID_pmos_part3_vgs1 = np.array([0, 0.001, 0.003, 0.005, 0.007, 0.007, 0.007, 0.005, 0.005, 0.005, 0.005])
ID_pmos_part3_vgs10 = np.array([0.008, 0.008, 0.008, 0.008, 0.008, 0.009, 0.012, 0.014, 0.017, 0.020, 0.022])

# Interpolate VGS=4V
ID_pmos_part3_vgs4 = ID_pmos_part3_vgs1 * (2/3) + ID_pmos_part3_vgs10 * (1/3)

ID_pmos_part3 = np.column_stack([ID_pmos_part3_vgs04, ID_pmos_part3_vgs1, ID_pmos_part3_vgs4, ID_pmos_part3_vgs10])

print("\nPMOS data loaded successfully with interpolated values")

## 3. Plot ID vs VGS at VDS = 10V with ION labeled (NMOS)

In [ ]:
# Extract ID at VDS=10V for NMOS
ID_at_VDS10_nmos = ID_nmos_part2[:, 3]  # Last column (VDS=10V)
ION_nmos = np.max(ID_at_VDS10_nmos)  # Maximum current = ION
ION_VGS_nmos = VGS_values_nmos[np.argmax(ID_at_VDS10_nmos)]

plt.figure(figsize=(10, 6))
plt.plot(VGS_values_nmos, ID_at_VDS10_nmos * 1000, 'b-o', linewidth=2, markersize=6, label='NMOS')
plt.plot(ION_VGS_nmos, ION_nmos * 1000, 'r*', markersize=15, label=f'$I_{{ON}}$ = {ION_nmos*1000:.1f} mA')

# Add annotation
plt.annotate(f'$I_{{ON}}$ = {ION_nmos*1000:.1f} mA', 
             xy=(ION_VGS_nmos, ION_nmos*1000), 
             xytext=(ION_VGS_nmos-2, ION_nmos*1000+10),
             arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
             fontsize=12, color='red')

plt.xlabel('$V_{GS}$ (V)', fontsize=13)
plt.ylabel('$I_D$ (mA)', fontsize=13)
plt.title('NMOS: $I_D$ vs $V_{GS}$ at $V_{DS}$ = 10V', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"NMOS ION = {ION_nmos*1000:.3f} mA at VGS = {ION_VGS_nmos} V")

## 4. Plot ID vs VGS at VDS = 10V with ION labeled (PMOS)

In [ ]:
# Extract ID at VDS=10V for PMOS
ID_at_VDS10_pmos = ID_pmos_part2[:, 3]  # Last column (VDS=10V)
ION_pmos = np.max(ID_at_VDS10_pmos)  # Maximum current = ION
ION_VGS_pmos = VGS_values_pmos[np.argmax(ID_at_VDS10_pmos)]

plt.figure(figsize=(10, 6))
plt.plot(VGS_values_pmos, ID_at_VDS10_pmos * 1000, 'r-o', linewidth=2, markersize=6, label='PMOS')
plt.plot(ION_VGS_pmos, ION_pmos * 1000, 'b*', markersize=15, label=f'$I_{{ON}}$ = {ION_pmos*1000:.1f} mA')

# Add annotation
plt.annotate(f'$I_{{ON}}$ = {ION_pmos*1000:.1f} mA', 
             xy=(ION_VGS_pmos, ION_pmos*1000), 
             xytext=(ION_VGS_pmos-2, ION_pmos*1000-3),
             arrowprops=dict(arrowstyle='->', color='blue', lw=1.5),
             fontsize=12, color='blue')

plt.xlabel('$V_{GS}$ (V)', fontsize=13)
plt.ylabel('$I_D$ (mA)', fontsize=13)
plt.title('PMOS: $I_D$ vs $V_{GS}$ at $V_{DS}$ = 10V', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"PMOS ION = {ION_pmos*1000:.3f} mA at VGS = {ION_VGS_pmos} V")

## 5. Plot gm vs VGS at VDS = 10V with gm(peak) labeled (NMOS)

In [ ]:
# Calculate transconductance gm = ΔID/ΔVGS for NMOS
gm_nmos = np.gradient(ID_at_VDS10_nmos, VGS_values_nmos)
gm_peak_nmos = np.max(gm_nmos)
gm_peak_VGS_nmos = VGS_values_nmos[np.argmax(gm_nmos)]

plt.figure(figsize=(10, 6))
plt.plot(VGS_values_nmos, gm_nmos * 1000, 'g-o', linewidth=2, markersize=6, label='NMOS $g_m$')
plt.plot(gm_peak_VGS_nmos, gm_peak_nmos * 1000, 'r*', markersize=15, 
         label=f'$g_{{m(peak)}}$ = {gm_peak_nmos*1000:.2f} mS')

# Add annotation
plt.annotate(f'$g_{{m(peak)}}$ = {gm_peak_nmos*1000:.2f} mS', 
             xy=(gm_peak_VGS_nmos, gm_peak_nmos*1000), 
             xytext=(gm_peak_VGS_nmos+1, gm_peak_nmos*1000+2),
             arrowprops=dict(arrowstyle='->', color='red', lw=1.5),
             fontsize=12, color='red')

plt.xlabel('$V_{GS}$ (V)', fontsize=13)
plt.ylabel('$g_m$ (mS)', fontsize=13)
plt.title('NMOS: Transconductance $g_m$ vs $V_{GS}$ at $V_{DS}$ = 10V', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"NMOS gm(peak) = {gm_peak_nmos*1000:.3f} mS at VGS = {gm_peak_VGS_nmos} V")

## 6. Plot gm vs VGS at VDS = 10V with gm(peak) labeled (PMOS)

In [ ]:
# Calculate transconductance gm = ΔID/ΔVGS for PMOS
gm_pmos = np.gradient(ID_at_VDS10_pmos, VGS_values_pmos)
gm_peak_pmos = np.max(gm_pmos)
gm_peak_VGS_pmos = VGS_values_pmos[np.argmax(gm_pmos)]

plt.figure(figsize=(10, 6))
plt.plot(VGS_values_pmos, gm_pmos * 1000, 'm-o', linewidth=2, markersize=6, label='PMOS $g_m$')
plt.plot(gm_peak_VGS_pmos, gm_peak_pmos * 1000, 'b*', markersize=15, 
         label=f'$g_{{m(peak)}}$ = {gm_peak_pmos*1000:.2f} mS')

# Add annotation
plt.annotate(f'$g_{{m(peak)}}$ = {gm_peak_pmos*1000:.2f} mS', 
             xy=(gm_peak_VGS_pmos, gm_peak_pmos*1000), 
             xytext=(gm_peak_VGS_pmos-2, gm_peak_pmos*1000-0.5),
             arrowprops=dict(arrowstyle='->', color='blue', lw=1.5),
             fontsize=12, color='blue')

plt.xlabel('$V_{GS}$ (V)', fontsize=13)
plt.ylabel('$g_m$ (mS)', fontsize=13)
plt.title('PMOS: Transconductance $g_m$ vs $V_{GS}$ at $V_{DS}$ = 10V', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print(f"PMOS gm(peak) = {gm_peak_pmos*1000:.3f} mS at VGS = {gm_peak_VGS_pmos} V")

## 7. Compute Threshold Voltage VT0 (NMOS)

In [ ]:
# Method 1: Linear extrapolation in saturation region
# Using sqrt(ID) vs VGS in saturation region (VDS=10V)

# Select points where device is in saturation (VGS > Vth, ID increasing)
# From data, current starts flowing around VGS=2-3V
sat_indices_nmos = (VGS_values_nmos >= 3) & (VGS_values_nmos <= 10)
VGS_sat_nmos = VGS_values_nmos[sat_indices_nmos]
ID_sat_nmos = ID_at_VDS10_nmos[sat_indices_nmos]

# Linear fit of sqrt(ID) vs VGS
sqrt_ID_sat_nmos = np.sqrt(ID_sat_nmos)
coeffs_nmos = np.polyfit(VGS_sat_nmos, sqrt_ID_sat_nmos, 1)
VT0_nmos = -coeffs_nmos[1] / coeffs_nmos[0]  # x-intercept

# Plot
plt.figure(figsize=(10, 6))
plt.plot(VGS_values_nmos, np.sqrt(ID_at_VDS10_nmos) * 1000, 'bo-', linewidth=2, markersize=6, label='Measured data')
VGS_fit = np.linspace(0, 10, 100)
sqrt_ID_fit = coeffs_nmos[0] * VGS_fit + coeffs_nmos[1]
plt.plot(VGS_fit, sqrt_ID_fit * 1000, 'r--', linewidth=2, label='Linear fit')
plt.axvline(VT0_nmos, color='g', linestyle=':', linewidth=2, label=f'$V_{{T0}}$ = {VT0_nmos:.2f} V')
plt.axhline(0, color='k', linestyle='-', linewidth=0.5)

plt.xlabel('$V_{GS}$ (V)', fontsize=13)
plt.ylabel('$\sqrt{I_D}$ (mA$^{1/2}$)', fontsize=13)
plt.title('NMOS: Threshold Voltage Extraction ($\sqrt{I_D}$ vs $V_{GS}$)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xlim(0, 10)
plt.tight_layout()
plt.show()

print(f"\nNMOS Threshold Voltage VT0 = {VT0_nmos:.3f} V")

## 8. Compute Threshold Voltage VT0 (PMOS)

In [ ]:
# For PMOS, select saturation region
sat_indices_pmos = (VGS_values_pmos >= 4) & (VGS_values_pmos <= 10)
VGS_sat_pmos = VGS_values_pmos[sat_indices_pmos]
ID_sat_pmos = ID_at_VDS10_pmos[sat_indices_pmos]

# Linear fit of sqrt(ID) vs VGS
sqrt_ID_sat_pmos = np.sqrt(ID_sat_pmos)
coeffs_pmos = np.polyfit(VGS_sat_pmos, sqrt_ID_sat_pmos, 1)
VT0_pmos = -coeffs_pmos[1] / coeffs_pmos[0]  # x-intercept

# Plot
plt.figure(figsize=(10, 6))
plt.plot(VGS_values_pmos, np.sqrt(ID_at_VDS10_pmos) * 1000, 'ro-', linewidth=2, markersize=6, label='Measured data')
VGS_fit_pmos = np.linspace(0, 10, 100)
sqrt_ID_fit_pmos = coeffs_pmos[0] * VGS_fit_pmos + coeffs_pmos[1]
plt.plot(VGS_fit_pmos, sqrt_ID_fit_pmos * 1000, 'b--', linewidth=2, label='Linear fit')
plt.axvline(VT0_pmos, color='g', linestyle=':', linewidth=2, label=f'$V_{{T0}}$ = {VT0_pmos:.2f} V')
plt.axhline(0, color='k', linestyle='-', linewidth=0.5)

plt.xlabel('$V_{GS}$ (V)', fontsize=13)
plt.ylabel('$\sqrt{I_D}$ (mA$^{1/2}$)', fontsize=13)
plt.title('PMOS: Threshold Voltage Extraction ($\sqrt{I_D}$ vs $V_{GS}$)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.xlim(0, 10)
plt.tight_layout()
plt.show()

print(f"\nPMOS Threshold Voltage VT0 = {VT0_pmos:.3f} V")

## 9. Plot ID vs VDS and Identify Breakdown Region (NMOS)

In [ ]:
# Plot ID vs VDS for different VGS values (NMOS)
plt.figure(figsize=(12, 7))

# Plot each VGS curve
colors = plt.cm.viridis(np.linspace(0, 1, len(VGS_values_nmos)))
for i, vgs in enumerate(VGS_values_nmos):
    if np.max(ID_nmos_part2[i, :]) > 0.001:  # Only plot if current is significant
        plt.plot(VDS_values_nmos, ID_nmos_part2[i, :] * 1000, 'o-', 
                linewidth=2, markersize=6, color=colors[i], label=f'$V_{{GS}}$ = {vgs} V')

# Add note about breakdown region
plt.text(8, 90, 'Breakdown region\n(if ID increases\nrapidly at high VDS)', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5), fontsize=10)

plt.xlabel('$V_{DS}$ (V)', fontsize=13)
plt.ylabel('$I_D$ (mA)', fontsize=13)
plt.title('NMOS: $I_D$ vs $V_{DS}$ Characteristics (Output Characteristics)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=9, loc='upper left')
plt.tight_layout()
plt.show()

print("Note: Breakdown typically occurs at high VDS where ID increases sharply.")
print("From the data, no clear breakdown is observed up to VDS=10V.")
print("Part 4 mentions VDS=14.25V - this might be near breakdown voltage.")

## 10. Plot ID vs VDS and Identify Breakdown Region (PMOS)

In [ ]:
# Plot ID vs VDS for different VGS values (PMOS)
plt.figure(figsize=(12, 7))

# Plot each VGS curve
colors = plt.cm.plasma(np.linspace(0, 1, len(VGS_values_pmos)))
for i, vgs in enumerate(VGS_values_pmos):
    if np.max(ID_pmos_part2[i, :]) > 0.001:  # Only plot if current is significant
        plt.plot(VDS_values_pmos, ID_pmos_part2[i, :] * 1000, 'o-', 
                linewidth=2, markersize=6, color=colors[i], label=f'$V_{{GS}}$ = {vgs} V')

# Add note about breakdown region
plt.text(7, 18, 'Breakdown region\n(if ID increases\nrapidly at high VDS)', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5), fontsize=10)

plt.xlabel('$V_{DS}$ (V)', fontsize=13)
plt.ylabel('$I_D$ (mA)', fontsize=13)
plt.title('PMOS: $I_D$ vs $V_{DS}$ Characteristics (Output Characteristics)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=9, loc='upper left')
plt.tight_layout()
plt.show()

print("Note: PMOS shows increasing current at higher VDS values for higher VGS.")
print("This could indicate approaching breakdown or strong channel-length modulation.")

## 11. Plot gd vs VDS and Extract λ (NMOS)

In [ ]:
# Calculate drain conductance gd = ΔID/ΔVDS for different VGS values (NMOS)
# Using data from Part 3 where we have VDS sweep at different VGS

plt.figure(figsize=(12, 7))

lambda_values_nmos = []

for i, vgs in enumerate(VGS_values_nmos_part3):
    ID_curve = ID_nmos_part3[:, i]
    gd = np.gradient(ID_curve, VDS_values_nmos_part3)
    
    if vgs >= 4:  # Only for strong inversion
        plt.plot(VDS_values_nmos_part3, gd * 1000, 'o-', linewidth=2, markersize=5, 
                label=f'$V_{{GS}}$ = {vgs} V')
        
        # Extract λ from saturation region (VDS > VGS - Vth)
        # In saturation: ID ≈ K(VGS-Vth)²(1 + λVDS)
        # gd = dID/dVDS ≈ K(VGS-Vth)² * λ
        # λ ≈ gd / ID (in saturation)
        
        sat_region = VDS_values_nmos_part3 >= 5
        if np.sum(sat_region) > 0:
            gd_sat = np.mean(gd[sat_region])
            ID_sat = np.mean(ID_curve[sat_region])
            if ID_sat > 0.001:
                lambda_val = gd_sat / ID_sat
                lambda_values_nmos.append(lambda_val)
                print(f"VGS={vgs}V: λ ≈ {lambda_val:.4f} V⁻¹")

plt.xlabel('$V_{DS}$ (V)', fontsize=13)
plt.ylabel('$g_d$ = $\Delta I_D / \Delta V_{DS}$ (mS)', fontsize=13)
plt.title('NMOS: Drain Conductance $g_d$ vs $V_{DS}$', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

if lambda_values_nmos:
    lambda_avg_nmos = np.mean(lambda_values_nmos)
    print(f"\nNMOS Average λ = {lambda_avg_nmos:.4f} V⁻¹")
    print(f"Channel-length modulation: 1/λ = {1/lambda_avg_nmos:.2f} V (Early voltage)")

## 12. Plot gd vs VDS and Extract λ (PMOS)

In [ ]:
# Calculate drain conductance gd = ΔID/ΔVDS for different VGS values (PMOS)

plt.figure(figsize=(12, 7))

lambda_values_pmos = []

for i, vgs in enumerate(VGS_values_pmos_part3):
    ID_curve = ID_pmos_part3[:, i]
    gd = np.gradient(ID_curve, VDS_values_pmos_part3)
    
    if vgs >= 4:  # Only for strong inversion
        plt.plot(VDS_values_pmos_part3, gd * 1000, 'o-', linewidth=2, markersize=5, 
                label=f'$V_{{GS}}$ = {vgs} V')
        
        # Extract λ from saturation region
        sat_region = VDS_values_pmos_part3 >= 5
        if np.sum(sat_region) > 0:
            gd_sat = np.mean(gd[sat_region])
            ID_sat = np.mean(ID_curve[sat_region])
            if ID_sat > 0.001:
                lambda_val = gd_sat / ID_sat
                lambda_values_pmos.append(lambda_val)
                print(f"VGS={vgs}V: λ ≈ {lambda_val:.4f} V⁻¹")

plt.xlabel('$V_{DS}$ (V)', fontsize=13)
plt.ylabel('$g_d$ = $\Delta I_D / \Delta V_{DS}$ (mS)', fontsize=13)
plt.title('PMOS: Drain Conductance $g_d$ vs $V_{DS}$', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

if lambda_values_pmos:
    lambda_avg_pmos = np.mean(lambda_values_pmos)
    print(f"\nPMOS Average λ = {lambda_avg_pmos:.4f} V⁻¹")
    print(f"Channel-length modulation: 1/λ = {1/lambda_avg_pmos:.2f} V (Early voltage)")

## Summary of Results

In [ ]:
print("="*60)
print("ECE335 Lab 3 - MOSFET Characterization Results Summary")
print("="*60)
print("\n--- NMOS Transistor ---")
print(f"1. ION (at VDS=10V): {ION_nmos*1000:.3f} mA")
print(f"2. gm(peak): {gm_peak_nmos*1000:.3f} mS at VGS={gm_peak_VGS_nmos} V")
print(f"3. Threshold Voltage VT0: {VT0_nmos:.3f} V")
if lambda_values_nmos:
    print(f"4. Channel-length modulation λ: {lambda_avg_nmos:.4f} V⁻¹")
    print(f"   Early Voltage VA = 1/λ: {1/lambda_avg_nmos:.2f} V")

print("\n--- PMOS Transistor ---")
print(f"1. ION (at VDS=10V): {ION_pmos*1000:.3f} mA")
print(f"2. gm(peak): {gm_peak_pmos*1000:.3f} mS at VGS={gm_peak_VGS_pmos} V")
print(f"3. Threshold Voltage VT0: {VT0_pmos:.3f} V")
if lambda_values_pmos:
    print(f"4. Channel-length modulation λ: {lambda_avg_pmos:.4f} V⁻¹")
    print(f"   Early Voltage VA = 1/λ: {1/lambda_avg_pmos:.2f} V")

print("\n" + "="*60)
print("Note: PMOS VDS=4V data was interpolated from VDS=1V and VDS=10V")
print("="*60)